In [233]:
import os
import pandas as pd
import numpy as np
import csv
from tqdm.notebook import tqdm
import time
from requests_html import HTMLSession
import re

In [59]:
# set up environment variable (depend on the user's account)

os.environ['SPOTIPY_CLIENT_ID']="3e3851d02b36446a85fec637a37c573f"
os.environ['SPOTIPY_CLIENT_SECRET']="9c763e54e63744a190c9085f64779680"

Dữ liệu được sử dụng trong đồ án này được lấy như sau:
- **Playlists**: lấy tất cả các public playlists của Spotify sử dụng Spotify API 
- **Tracks**: thông tin sơ lược về tracks và kết quả phân tích audio của tracks được lấy sử dụng Spotify API 
- **Artists**: kết hợp cả Spotify API và parse HTML để lấy được nhiều thông tin nhất có thể (có những thông tin **không thể** lấy được nếu dùng API như số người nghe mỗi tháng `monthly_listeners`)


# 1. Dùng Spotify API để crawl dữ liệu playlist và track 

In [60]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [61]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

## a. Crawl playlist data

In [62]:
def crawl_playlist_data(user='spotify', num_playlist=50, replace=True):
    playlist_attributes = [
        'playlist_id',
        'playlist_name',
        'description',
        'num_tracks',
        'num_followers'
    ]

    track_attributes = [
        'track_id',
        'track_name',
        'playlist_id',
        'playlist_name',
        'artist_ids',
        'artist_names',
        'album_id',
        'album_name',
        'track_duration_ms'
    ]
    
    if replace:
        # write header files
        with open('../../data/playlists.tsv', 'w') as f:
            f.write('\t'.join(playlist_attributes) + '\n')

        with open('../../data/tracks.tsv', 'w') as f:
            f.write('\t'.join(track_attributes) + '\n')

    # crawl data     
    i = 0
    for i in tqdm(range(0, num_playlist, 50), desc="All playlists"):
        time.sleep(0.1)
        playlists = sp.user_playlists(user, offset=i)

        playlist_data = {}
        for attr in playlist_attributes:
            playlist_data[attr] = []

        track_data = {}
        for attr in track_attributes:
            track_data[attr] = []

        for pl in tqdm(playlists['items'], desc='Playlist {} to {}'.format(i, i+50)):
            try:
                playlist = sp.playlist(pl['id'])

                # add playlist information to playlist data 
                playlist_data['playlist_id'].append(playlist['id'])
                playlist_data['playlist_name'].append(playlist['name'])
                playlist_data['description'].append(playlist['description'])
                playlist_data['playlist_name'].append(playlist['name'])
                playlist_data['num_tracks'].append(playlist['tracks']['total'])
                playlist_data['num_followers'].append(playlist['followers']['total'])

                # add track information to track data 
                for track in playlist['tracks']['items']:
                    track_data['playlist_id'].append(playlist['id'])
                    track_data['playlist_name'].append(playlist['name']) 
                    track = track['track']
                    track_data['track_id'].append(track['id'])
                    track_data['track_name'].append(track['name'])         
                    track_data['artist_ids'].append([artist['id'] for artist in track['artists']])
                    track_data['artist_names'].append([artist['name'] for artist in track['artists']])
                    track_data['album_id'].append(track['album']['id'])
                    track_data['album_name'].append(track['album']['name'])
                    track_data['track_duration_ms'].append(track['duration_ms'])
            except:
                print('Error while processing the', i, 'playlist:', pl['external_urls']['spotify'])
                
            i += 1

        with open('../../data/playlists.tsv', 'a') as f:
            for i in range(len(playlist_data['playlist_id'])):
                f.write('\t'.join([str(playlist_data[k][i]).strip().replace('\t', ' ').replace('\n', ' ').replace('\r', ' ') 
                                   for k in playlist_attributes])
                                   + '\n')

        with open('../../data/tracks.tsv', 'a') as f:
            for i in range(len(track_data['track_id'])):
                f.write('\t'.join([str(track_data[k][i]).strip().replace('\t', ' ').replace('\n', ' ').replace('\r', ' ') 
                                   for k in track_attributes])
                                   + '\n')        

In [6]:
crawl_playlist_data(num_playlist=2000)

Error while processing the 36 playlist: https://open.spotify.com/playlist/37i9dQZF1DWT2jS7NwYPVI



Error while processing the 133 playlist: https://open.spotify.com/playlist/37i9dQZF1DWUq3wF0JVtEy



Error while processing the 223 playlist: https://open.spotify.com/playlist/37i9dQZF1DWYHkSks5rujC
Error while processing the 225 playlist: https://open.spotify.com/playlist/37i9dQZF1DWZreqadA03A8
Error while processing the 226 playlist: https://open.spotify.com/playlist/37i9dQZF1DXdwF9DytRe8w
Error while processing the 234 playlist: https://open.spotify.com/playlist/37i9dQZF1DXec50AjHrNTq



Error while processing the 251 playlist: https://open.spotify.com/playlist/37i9dQZF1DWYrlaUEYy4Vg



Error while processing the 396 playlist: https://open.spotify.com/playlist/37i9dQZF1DX56qfiUZBncF



...

Error while processing the 1054 playlist: https://open.spotify.com/playlist/37i9dQZF1DWWOGXILUAh53
Error while processing the 1088 playlist: https://open.spotify.com/playlist/37i9dQZF1DXcb6CQIjdqKy



Error while processing the 1175 playlist: https://open.spotify.com/playlist/37i9dQZF1DXcnkReojaCnV



Error while processing the 1239 playlist: https://open.spotify.com/playlist/37i9dQZF1DX0sQWfevMRw3



Error while processing the 1320 playlist: https://open.spotify.com/playlist/37i9dQZF1DWXfgo3OOonqa



Error while processing the 1398 playlist: https://open.spotify.com/playlist/37i9dQZF1DXdc0DUqaW3MZ



## b. Crawl track audio information

In [63]:
track_df = pd.read_csv('../../data/tracks.tsv', sep='\t')
track_df.head()

,track_id,track_name,playlist_id,playlist_name,artist_ids,artist_names,album_id,album_name,track_duration_ms
0,463CkQjx2Zk1yXoBuierM9,Levitating (feat. DaBaby),37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,"['6M2wZ9GZgrQXHCFfjv46we', '4r63FhuTkUYltbVAg5...","['Dua Lipa', 'DaBaby']",04m06KhJUuwe1Q487puIud,Levitating (feat. DaBaby),203064
1,7igeByaBM0MgGsgXtNxDJ7,positions,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],3jqEvfiu2ENgmgzZq27zbi,positions,172324
2,0lx2cLdOt3piJbcaXIV74f,willow,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,['06HL4z0CvFAxyc27GXpf02'],['Taylor Swift'],2Xoteh7uEpea4TohMxjtaq,evermore,214706
3,2Z8yfpFX0ZMavHkcIeHiO1,Monster (Shawn Mendes & Justin Bieber),37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,"['7n2wHs1TKAczGzO7Dd2rGr', '1uNFoZAHBGtllmzznp...","['Shawn Mendes', 'Justin Bieber']",3yVVL2EYLp8g7gT08VvYKy,Monster,178994
4,6zFMeegAMYQo0mt8rXtrli,HOLIDAY,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,['7jVv8c5Fj3E9VhNjxT4snq'],['Lil Nas X'],4EvukZrmNBiqJbs3LwOSHu,HOLIDAY,154997


In [127]:
def crawl_audio_data(track_file='../../data/tracks.tsv'):
    track_df = pd.read_csv(track_file, sep='\t')
    track_ids = list(set(track_df['track_id']))
    audio_attributes = [
        'id',
        'danceability',
        'energy',
        'key',
        'loudness',
        'mode',
        'speechiness',
        'acousticness',
        'instrumentalness',
        'liveness',
        'valence',
        'tempo',
        'analysis_url',
        'time_signature'
    ]
    
    track_attributes = [
        'available_markets',
        'popularity'
    ]

    with open('../../data/audio_features.tsv', 'w') as f:
        f.write('\t'.join(audio_attributes + track_attributes) + '\n')
    
    i = -1
    for start in tqdm(range(0, len(track_ids), 50), desc='All tracks'):
        time.sleep(0.1)
        data = {}
        for attr in audio_attributes + track_attributes:
            data[attr] = []
        
        end = min(start+50, len(track_ids))
        audio_features = sp.audio_features(track_ids[start:end])
        track_features = sp.tracks(track_ids[start:end])
        for audio, track in zip(audio_features, track_features['tracks']):
            i += 1
            try:
                for attr in audio_attributes:
                    data[attr].append(audio[attr])
                    
                for attr in track_attributes:
                    data[attr].append(track[attr])
            except:
                print('Error while processing the {}th track with id {}'.format(i, track_ids[i]))
                                
        with open('../../data/audio_features.tsv', 'a') as f:
            for i in range(len(data['id'])):
                f.write('\t'.join([str(data[k][i]).strip().replace('\t', ' ').replace('\n', ' ').replace('\r', ' ') 
                                   for k in (audio_attributes+track_attributes)])
                                   + '\n')              

In [128]:
crawl_audio_data()

Error while processing the 94th track with id 6tvzakkt80JJBQDwv7gOJ2
Error while processing the 84th track with id 1bCmU5Ouw4G0l9d73oNxFc
Error while processing the 80th track with id 7aaHlvqJoea0S46e3YPVEX
Error while processing the 65th track with id 6POfrpc0tv287bg8lpfsRs
Error while processing the 64th track with id 3tn802xiWFEfwoBnFfORTH
Error while processing the 86th track with id 3kgx7Hw9C0cMPOYUdtIrXI
Error while processing the 66th track with id 0VeJBq4WnjbpUlxew0zOjp
Error while processing the 85th track with id 2uqowBnUT4KrCszpKiOLCH
Error while processing the 72th track with id 0zEswgpayNg0tlsr2L0q7P
Error while processing the 84th track with id 1bCmU5Ouw4G0l9d73oNxFc
Error while processing the 75th track with id 422kCwhe9nDgNDcYHdmsEl
Error while processing the 54th track with id 7HQ8kBRQCdhRtCBwu8a4a7
Error while processing the 56th track with id 1Oewmmr12xuhgzvUJ9Q0cf
Error while processing the 70th track with id 1MIQyiSyJcRBKg3pCLESFa
Error while processing the 55th tr

# 2. Dùng Spotify API kết hợp Parse HTML để lấy thông tin artist 

In [170]:
track_df = pd.read_csv('../../data/tracks.tsv', sep='\t')
track_df.head()

,track_id,track_name,playlist_id,playlist_name,artist_ids,artist_names,album_id,album_name,track_duration_ms
0,463CkQjx2Zk1yXoBuierM9,Levitating (feat. DaBaby),37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,"['6M2wZ9GZgrQXHCFfjv46we', '4r63FhuTkUYltbVAg5...","['Dua Lipa', 'DaBaby']",04m06KhJUuwe1Q487puIud,Levitating (feat. DaBaby),203064
1,7igeByaBM0MgGsgXtNxDJ7,positions,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],3jqEvfiu2ENgmgzZq27zbi,positions,172324
2,0lx2cLdOt3piJbcaXIV74f,willow,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,['06HL4z0CvFAxyc27GXpf02'],['Taylor Swift'],2Xoteh7uEpea4TohMxjtaq,evermore,214706
3,2Z8yfpFX0ZMavHkcIeHiO1,Monster (Shawn Mendes & Justin Bieber),37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,"['7n2wHs1TKAczGzO7Dd2rGr', '1uNFoZAHBGtllmzznp...","['Shawn Mendes', 'Justin Bieber']",3yVVL2EYLp8g7gT08VvYKy,Monster,178994
4,6zFMeegAMYQo0mt8rXtrli,HOLIDAY,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,['7jVv8c5Fj3E9VhNjxT4snq'],['Lil Nas X'],4EvukZrmNBiqJbs3LwOSHu,HOLIDAY,154997


In [ ]:
a = sp.artists(artist_ids[0:])

In [ ]:
def crawl_artist_data(track_file='../../data/tracks.tsv'):
    # create HTML Session
    session = HTMLSession()
    
    track_df = pd.read_csv(track_file, sep='\t')
    artist_ids = track_df['artist_ids'].apply(lambda x: x[1:-1].split(', '))
    artist_ids = list(artist_id[1:-1] for artist_id in set(artist_ids.explode().values))
    
    artist_attributes = [
        'id',
        'name',
        'popularity',
        'genres',
        'num_followers',
        'monthly_listeners'
    ]

    with open('../../data/artists.tsv', 'w') as f:
        f.write('\t'.join(artist_attributes) + '\n')
    
    i = -1
    for start in tqdm(range(0, len(artist_ids), 25), desc='All artists'):
        time.sleep(0.1)
        data = {}
        for attr in artist_attributes:
            data[attr] = []
        
        end = min(start+50, len(artist_ids))
        artist_features = sp.artists(artist_ids[start:end])
        for artist in artist_features['artists']:
            i += 1
            
            if artist is None:
                continue
                
            url = artist['external_urls']['spotify']
            monthly_listeners = 0
            
            # get HTML content
            content = session.get(url).text
            
            # use regular expression to find the value of monthly listeners
            text = r"Monthly Listeners: \d+"
            res = re.findall(text, content)
            for r in res:
                try:
                    # update value if available and break the loop
                    monthly_listeners = int(r.split(': ')[-1])
                    break 
                except:
                    pass
                
            try:    
                
                for attr in artist_attributes[:-2]:
                    data[attr].append(artist[attr])
                data['monthly_listeners'].append(monthly_listeners)
                data['num_followers'].append(artist['followers']['total'])
            except:
                print('Error while processing the {}th artist with id {}'.format(i, artist_ids[i]))
                                
        with open('../../data/artists.tsv', 'a') as f:
            for i in range(len(data['id'])):
                f.write('\t'.join([str(data[k][i]).strip().replace('\t', ' ').replace('\n', ' ').replace('\r', ' ') 
                                   for k in artist_attributes])
                                   + '\n')              

**LƯU Ý:** Crawl data nghệ sĩ do có parse HTML nên sẽ tốn nhiều thời gian hơn so với việc crawl playlist và track data. Việc crawl data nghệ sĩ có thể tốn khoảng **5.5 tiếng** tùy cấu hình máy.

In [279]:
crawl_artist_data()